# Data Definition Language

## Table Management

### 🔸 PostgreSQL Data Types

| Type          | Description                                  | Example                         |
|---------------|----------------------------------------------|----------------------------------|
| `SMALLINT`    | Small integers (-32k to 32k)                 | `age SMALLINT`                  |
| `INTEGER`     | Standard integers                            | `count INTEGER`                 |
| `SERIAL`      | Auto-incrementing integer                    | `id SERIAL PRIMARY KEY`         |
| `VARCHAR(n)`  | Variable-length string with limit            | `email VARCHAR(100)`            |
| `CHAR(n)`     | Fixed-length string                          | `country_code CHAR(2)`          |
| `TEXT`        | Unlimited-length string                      | `description TEXT`              |
| `BOOLEAN`     | TRUE, FALSE, NULL                            | `is_active BOOLEAN`             |
| `NUMERIC(p,s)`| Exact decimal (e.g. money)                   | `price NUMERIC(10, 2)`          |
| `DATE`        | Calendar date                                | `birth_date DATE`               |
| `TIMESTAMP`   | Date and time                                | `created_at TIMESTAMP`          |
| `UUID`        | Unique identifier                            | `token UUID`                    |
| `JSONB`       | Binary JSON storage                          | `settings JSONB`                |
| `ARRAY`       | PostgreSQL arrays                            | `tags TEXT[]`                   |

In [ ]:
CREATE TYPE status AS ENUM ('active', 'inactive', 'banned');

### 🔸 Creating Tables with Constraints

In [ ]:
CREATE TABLE employee (
  id SERIAL PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email TEXT UNIQUE,
  department_id INT REFERENCES department(id)
);

### 🔸 NULL vs NOT NULL
- `NOT NULL`: must be filled (e.g., critical fields like `email`).
- `NULL`: optional (e.g., `comment`).

### 🔸 ALTER TABLE

In [ ]:
ALTER TABLE employee ADD COLUMN salary NUMERIC;

In [ ]:
ALTER TABLE employee ALTER COLUMN salary TYPE INTEGER;

In [ ]:
ALTER TABLE employee DROP COLUMN salary;

### 🔸 RENAME TABLE / COLUMN

In [ ]:
ALTER TABLE employee RENAME TO staff;

In [ ]:
ALTER TABLE staff RENAME COLUMN name TO full_name;

## Views and Virtual Tables

**Why use views?**
- 🧩 Abstract complexity
- 🔐 Enforce access control
- 🔄 Reusability for reporting or repeated queries

In [ ]:
CREATE VIEW active_customers AS
SELECT customer_id, first_name, last_name
FROM customer
WHERE active = TRUE;

In [ ]:
SELECT * FROM active_customers;

In [ ]:
DROP VIEW active_customers;

## Logical Delete (Soft Delete)
Instead of physically removing data, mark it as deleted using a flag column like `is_deleted`.

In [ ]:
ALTER TABLE customer ADD COLUMN is_deleted BOOLEAN DEFAULT FALSE;

In [ ]:
UPDATE customer SET is_deleted = TRUE WHERE customer_id = 12;

## Self-Relations: Hierarchies and Recursion
A self-relation is when a row relates to another row in the **same table**, like employees and their managers.

In [ ]:
CREATE TABLE employee (
  id SERIAL PRIMARY KEY,
  name TEXT,
  manager_id INT REFERENCES employee(id)
);

### 🔸 Recursive Query (WITH RECURSIVE)
Query a chain of management hierarchy using Common Table Expressions.

In [ ]:
WITH RECURSIVE management_chain AS (
  SELECT id, name, manager_id FROM employee WHERE manager_id IS NULL
  UNION ALL
  SELECT e.id, e.name, e.manager_id
  FROM employee e
  JOIN management_chain mc ON e.manager_id = mc.id
)
SELECT * FROM management_chain;

## Indexes

### 🔸 Why Use Indexes?
- Speed up search & joins
- Reduce full table scans

In [ ]:
CREATE INDEX idx_customer_email ON customer(email);

In [ ]:
EXPLAIN ANALYZE SELECT * FROM customer WHERE email = 'x@x.com';

## Trigger: Run logic automatically when data changes

In [ ]:
CREATE OR REPLACE FUNCTION update_timestamp()
RETURNS TRIGGER AS $$
BEGIN
  NEW.updated_at = NOW();
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER set_updated_at
BEFORE UPDATE ON customer
FOR EACH ROW
EXECUTE FUNCTION update_timestamp();

## Function: Reusable SQL logic that returns a result

In [ ]:
CREATE OR REPLACE FUNCTION active_customer_count()
RETURNS INTEGER AS $$
BEGIN
  RETURN (SELECT COUNT(*) FROM customer WHERE active = TRUE);
END;
$$ LANGUAGE plpgsql;

## Procedure: Executes a sequence of steps, often without returning a value

In [ ]:
CREATE PROCEDURE cleanup_logs()
LANGUAGE SQL
AS $$
  DELETE FROM logs WHERE created_at < NOW() - INTERVAL '30 days';
$$;

CALL cleanup_logs();

### Comparison Table
| Feature     | Trigger                         | Function                      | Procedure                     |
|-------------|----------------------------------|-------------------------------|-------------------------------|
| **When**    | Auto-run on INSERT/UPDATE/DELETE| Called in SQL (`SELECT`)     | Called with `CALL`            |
| **Returns** | VOID or `NEW`/`OLD` row         | Scalar or Table               | Usually VOID                  |
| **Use case**| Auto-maintenance                | Calculations, reuse logic     | Batch updates, cleanup jobs   |

## Normalization

### 🔸 Step-by-step Normalization Example
**Unnormalized Table:**

In [ ]:
CREATE TABLE orders (
  order_id INT,
  customer_name TEXT,
  items TEXT -- 'pen, paper, notebook'
);

**1NF:** Remove repeated items

In [ ]:
CREATE TABLE order_item (
  order_id INT,
  item TEXT
);

**2NF:** Remove partial dependencies

In [ ]:
CREATE TABLE customer (
  id SERIAL PRIMARY KEY,
  name TEXT
);
ALTER TABLE orders ADD COLUMN customer_id INT REFERENCES customer(id);

**3NF:** Remove transitive dependencies

In [ ]:
CREATE TABLE item (
  id SERIAL PRIMARY KEY,
  name TEXT,
  price NUMERIC
);
CREATE TABLE order_item (
  order_id INT,
  item_id INT REFERENCES item(id)
);

## Python psycopg2 and ORMs

In [ ]:
import psycopg2
conn = psycopg2.connect(
  dbname='dvdrental', user='postgres', password='yourpass', host='localhost')
cur = conn.cursor()
cur.execute('SELECT * FROM customer LIMIT 5;')
for row in cur.fetchall():
    print(row)
cur.close()
conn.close()